# Data exploration 

---

Group name: O

---


## Introduction

*This section includes a short description of the data* 

## Setup

In [10]:
import pandas as pd
import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Data

## Import data

In [11]:
DATA = "../data/external/nba_elo_latest.csv"

### Data structure

In [12]:
df = pd.read_csv(DATA)
print(df)

df.info()

            date  season  neutral  playoff team1 team2     elo1_pre  \
0     2022-10-18    2023        0      NaN   BOS   PHI  1657.639749   
1     2022-10-18    2023        0      NaN   GSW   LAL  1660.620307   
2     2022-10-19    2023        0      NaN   IND   WAS  1399.201934   
3     2022-10-19    2023        0      NaN   DET   ORL  1393.525172   
4     2022-10-19    2023        0      NaN   MEM   NYK  1605.024654   
...          ...     ...      ...      ...   ...   ...          ...   
1225  2023-04-09    2023        0      NaN   OKC   MEM  1418.664795   
1226  2023-04-09    2023        0      NaN   LAL   UTA  1404.345291   
1227  2023-04-09    2023        0      NaN   POR   GSW  1426.624444   
1228  2023-04-09    2023        0      NaN   PHO   LAC  1596.463751   
1229  2023-04-09    2023        0      NaN   DEN   SAC  1560.693716   

         elo2_pre  elo_prob1  elo_prob2  ...  carm-elo2_post  raptor1_pre  \
0     1582.247327   0.732950   0.267050  ...             NaN  1693.243

### Data corrections

In [13]:
df['score1'] = df['score1'].astype("category")
df['score2'] = df['score2'].astype("category")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   date            1230 non-null   object  
 1   season          1230 non-null   int64   
 2   neutral         1230 non-null   int64   
 3   playoff         0 non-null      float64 
 4   team1           1230 non-null   object  
 5   team2           1230 non-null   object  
 6   elo1_pre        1230 non-null   float64 
 7   elo2_pre        1230 non-null   float64 
 8   elo_prob1       1230 non-null   float64 
 9   elo_prob2       1230 non-null   float64 
 10  elo1_post       204 non-null    float64 
 11  elo2_post       204 non-null    float64 
 12  carm-elo1_pre   0 non-null      float64 
 13  carm-elo2_pre   0 non-null      float64 
 14  carm-elo_prob1  0 non-null      float64 
 15  carm-elo_prob2  0 non-null      float64 
 16  carm-elo1_post  0 non-null      float64 
 17  carm-elo2_post

### Variable lists

In [16]:
# make a list of variables you want to use
var_list = ['score1']

In [22]:
source = df[var_list]

In [23]:
source

,score1
0,126.0
1,123.0
2,107.0
3,113.0
4,115.0
...,...
1225,NaN
1226,NaN
1227,NaN
1228,NaN


In [24]:
#Ich möchte die NaN Werte entfernen

SourceClean =source.dropna(how='all')

In [25]:
SourceClean

,score1
0,126.0
1,123.0
2,107.0
3,113.0
4,115.0
...,...
199,105.0
200,113.0
201,126.0
202,106.0


In [ ]:
alt.Chart(source).mark_bar().encode(
    x='homeownership',
    y=alt.Y('count(homeownership)')
)